In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
from scipy.signal import welch

## Load Files

In [ ]:
data_desk_arr = np.load("data_desk.npy")
print("Desk Array shape:", data_desk_arr.shape)

data_hand_arr = np.load("data_hand.npy")
print("Hand Array shape:", data_hand_arr.shape)

## Diff Matrix

In [ ]:
# Get the shape of the loaded array
num_frames, ROI_H, ROI_W, colors = data_hand_arr.shape

# Create a new array with one less frame
diff_arr = np.zeros((num_frames - 1, ROI_H, ROI_W, colors), dtype=data_hand_arr.dtype)
print(diff_arr.shape)  # should be (num_frames - 1, ROI_H, ROI_W, 2)

In [ ]:
for c in range(colors):
    for f in range(num_frames - 1): # skip last one
        for h in range(ROI_H):
            for w in range(ROI_W):
                point_i = data_hand_arr[f, h, w, c]
                point_n = data_hand_arr[f+1, h, w, c]

                diff = point_n -point_i
                diff_arr[f, h, w, c] = diff

In [ ]:
for c in range(colors):
    for f in range(num_frames - 1): # skip last one
        # for h in range(ROI_H):
            # for w in range(ROI_W):

                print(f"diff_arr[{f}, {1}, {1}, {c}]: {diff_arr[f, 1, 1, c]}")

## FFT

In [ ]:
# Compute FFT along the time axis (axis=0) for all pixels and channels
fft_arr = np.fft.fft(data_hand_arr, axis=0)

print("FFT shape:", fft_arr.shape)

In [ ]:
# Choose pixel and channel
y, x, c = 10, 20, 0  # row 10, col 20, R channel

# Extract time series for that pixel
pixel_signal = data_hand_arr[:, y, x, c]

# Compute spectrogram
# nperseg controls the segment length; adjust for resolution
frequencies, times, Sxx = spectrogram(pixel_signal, nperseg=16)

# Plot
plt.figure(figsize=(8, 4))
plt.pcolormesh(times, frequencies, Sxx, shading='gouraud', cmap='viridis')
plt.ylabel('Frequency [frames^-1]')
plt.xlabel('Time [frames]')
plt.title(f"Spectrogram for pixel ({y},{x}), channel {c}")
plt.colorbar(label='Power')
plt.show()

In [ ]:
# Choose pixel and channel
y, x, c = 10, 20, 0  # row 10, col 20, R channel

# Extract time series for that pixel
pixel_signal = data_hand_arr[:, y, x, c]

# Compute Power Spectral Density using Welch's method
# fs is the sampling frequency; if frames are uniform, fs=1 frame/unit
frequencies, psd = welch(pixel_signal, fs=10, nperseg=min(256, len(pixel_signal)))

# Plot PSD
plt.figure(figsize=(8, 4))
plt.semilogy(frequencies, psd)
plt.xlabel("Frequency [frames⁻¹]")
plt.ylabel("Power Spectral Density")
plt.title(f"PSD for pixel ({y},{x}), channel {c}")
plt.grid(True)
plt.show()